In [72]:
# Given a list of audit entries. Determine field changes from one row to the next
import pandas as pd

In [84]:
df1 = pd.read_csv('transaction_audit_in.csv')
df1

,transaction_id,account_number,amount,status,last_action,last_modified_timestamp
0,1,12345,10.0,pending,I,2023-01-01 00:00:00
1,1,12345,20.5,pending,U,2023-01-01 12:00:00
2,1,12345,20.2,posted,U,2023-01-01 12:30:00
3,2,99999,23.8,pending,I,2023-01-01 00:00:00


In [79]:
df_transaction = df1[df1['transaction_id'] == 1].sort_values(by=['last_modified_timestamp'], ascending=True)
df_transaction = df_transaction.melt(id_vars=['transaction_id', 'account_number','last_action', 'last_modified_timestamp'],
    var_name='field_name', value_name='field_value')
df_transaction.sort_values(by=['field_name', 'last_modified_timestamp'], ascending=True)

,transaction_id,account_number,last_action,last_modified_timestamp,field_name,field_value
0,1,12345,I,2023-01-01 00:00:00,amount,10.0
1,1,12345,U,2023-01-01 12:00:00,amount,20.5
2,1,12345,U,2023-01-01 12:30:00,amount,20.2


In [80]:
df_transaction["field_value_previous"] = df_transaction.groupby(['field_name'])["field_value"].shift(1).fillna('N/A')
df_transaction

,transaction_id,account_number,last_action,last_modified_timestamp,field_name,field_value,field_value_previous
0,1,12345,I,2023-01-01 00:00:00,amount,10.0,N/A
1,1,12345,U,2023-01-01 12:00:00,amount,20.5,10.0
2,1,12345,U,2023-01-01 12:30:00,amount,20.2,20.5


In [81]:
df_transaction.to_csv('transaction_audit_out.csv', index=False)

In [82]:
df_updates = df_transaction[(df_transaction['field_value_previous'] != 'N/A') & (df_transaction['field_value'] != df_transaction['field_value_previous'])]
df_updates

,transaction_id,account_number,last_action,last_modified_timestamp,field_name,field_value,field_value_previous
1,1,12345,U,2023-01-01 12:00:00,amount,20.5,10.0
2,1,12345,U,2023-01-01 12:30:00,amount,20.2,20.5


In [83]:
df_updates.to_csv('transaction_audit_delta_out.csv', index=False)